小学校の在籍人数を地図上にマッピングする（Folium、GeoPandas利用）

https://qiita.com/Gyutan/items/f9b365d88a1ff89671f0

行いたいこと

（1）学校の位置を地図上に示す →経度緯度と学校名のリストがあればよい

（2）学校の校区を示す →ポリゴン操作が必要

（3）各学校の在籍人数を示す →階層プロットが必要

なお対象地域は、埼玉県相模原市とする。


1. ベースになる地図を表示する。

2. 1の地図の上に位置情報を重ねる。
    
- 小学校の位置など
    
3. 地図上の各オフジェクトに属性情報を重ねる
　　　　　
- 在校生の人数など


In [ ]:
#必要なモジュールのインポート
import json
from pathlib import Path

import folium
from folium import plugins
print( "folium version is {}".format(folium.__version__) )

import geopandas as gpd
import pandas as pd

import matplotlib.pyplot as plt

東京都の小学校校区をタウンロードする(A27-16_13_GML.zip)

https://nlftp.mlit.go.jp/ksj/gml/datalist/KsjTmplt-A27-v2_1.html


このファイルを展開すると， shapeフォルダの中に，A27-16_13.shpと，A27P-16_13.shpがある。Pの付いた方は，小学校の所在地点を表し，ついていない方は，校区を表す。


In [ ]:
elschool="data/A27-16_13_GML/shape/A27-16_13.shp"
df_e=gpd.read_file(elschool,encoding="shift-jis")
df_e.columns

各列の内容は次のとおりである。

A27_005: 市区町村コード

A27_006: 設置主体

A27_007: 小学校の名称

A27_008: 小学校の設置所在地

In [ ]:
#町田市立の各小学校の校区
machida_p=df_e[df_e["A27_006"]=="町田市立"]


In [ ]:
machida_p.plot(figsize=(40,40))

In [ ]:
machida_p=machida_p.reset_index(drop=True)

In [ ]:
machida_p=machida_p.sort_values("A27_007")
machida_p=machida_p.reset_index(drop=True)
machida_p

In [ ]:
elschool_p="data/A27-16_13_GML/shape/A27P-16_13.shp"
df_ep=gpd.read_file(elschool_p,encoding="Shift-JIS")
df_ep.columns

各列の内容は次のとおりである。

A27_001: 市区町村コード

A27_002: 小学校の設置主体

A27_003: 小学校の名称

A27_004: 小学校の設置所在地

In [ ]:
machida_ep=df_ep[df_ep["A27_002"]=="町田市立"]
machida_ep=machida_ep.reset_index(drop=True)
machida_ep.plot()

小学校の所在地の緯度と経度は，それぞれdf_ep['geometry'].yとdf_ep['geometry'].xに記録されている。

In [ ]:
df_ep['geometry'].y,df_ep['geometry'].x

In [ ]:
#地図に位置をマッピングします。
map_center = [35.546559,139.438527]  #町田市役所
m2_machida = folium.Map(location=map_center, tiles='openstreetmap', zoom_start=13)
for i, dt in machida_ep.iterrows(): #町田市立の各小学校に対して
    folium.Marker(location=[dt['geometry'].y,dt['geometry'].x], popup='{},{}'.format(dt['A27_002'],dt['A27_003'])).add_to(m2_machida)
m2_machida.save('mashida_e_school.html')

In [ ]:
display(m2_machida)

In [ ]:
map_center = [35.546559,139.438527]  #町田市役所
m_machida_sh = folium.Map(location=map_center, tiles='cartodbpositron', zoom_start=13)
#校区を重ねる
folium.Choropleth(geo_data=machida_p.to_json()).add_to(m_machida_sh)
#所在地を重ねる
for i, dt in machida_ep.iterrows(): #町田市立の各小学校に対して
    folium.Marker(location=[dt['geometry'].y,dt['geometry'].x], popup='{},{}'.format(dt['A27_002'],dt['A27_003'])).add_to(m_machida_sh)

In [ ]:
display(m_machida_sh)

この地図に，生徒数と学級数の情報を載せる。これらの情報は，町田市のホームページから取得する。

https://kosodate-machida.tokyo.jp/soshiki/5/3/239.html


In [ ]:
machida_ep=machida_ep.sort_values("A27_003")
machida_ep=machida_ep.reset_index(drop=True)
machida_ep

この地図に，生徒数と学級数の情報を載せる。これらの情報は，町田市のホームページから取得する。

https://kosodate-machida.tokyo.jp/soshiki/5/3/239.html

このページからダウンロードしたエクセルファイルから，小学校ごとに，各学年の児童数，学級数のデータを抜き出し，csvファイル(machida_ele_count.csv)として保存する。そして，そのcsvファイルをDataFrameとして読み込む。


In [ ]:
machida_ele_count_fp="data/machida_ele_count.csv"
machida_ele_count=pd.read_csv(machida_ele_count_fp)

In [ ]:
machida_ele_count=machida_ele_count.sort_values("学校名")
machida_ele_count=machida_ele_count.reset_index(drop=True)
machida_ele_count

In [ ]:
map_center = [35.546559,139.438527]  #町田市役所
m_machida_sh = folium.Map(location=map_center, tiles='cartodbpositron', zoom_start=13)
#校区を重ねる
folium.Choropleth(geo_data=machida_p.to_json()).add_to(m_machida_sh)
#所在地を重ねる
for i, dt in machida_ep.iterrows(): #町田市立の各小学校に対して
    folium.Marker(location=[dt['geometry'].y,dt['geometry'].x], popup='{},{}'.format(dt['A27_002'],dt['A27_003'])).add_to(m_machida_sh)

folium.Choropleth(geo_data=machida_ep.to_json(), # GeoJSONファイル
#                        name = 'choropleth', # map名
                        data = machida_ele_count,
                        columns=['id', '計児童・生徒数'], # 点数データのkey列とvalue列を指定
                       key_on='features.properties.A27_007', # GeoJSONファイル内のキーを指定
                       fill_opacity=0.7, 
                        line_opacity=0.2,
                        line_color='red',
                  threshold_scale=[0,100,200,300,400,500,600,700,800,900]
    ).add_to(m_machida_sh)
#LayerControl().add_to(m)
#folium.Choropleth(geo_data=machida_p.to_json()).add_to(m_machida_sh)

In [ ]:
display(m_machida_sh)

In [ ]:
machida_p.to_json()
machida_p.to_file("machida_p.geojson", driver='GeoJSON')

In [ ]:
machida_p

In [ ]:
machida_p["A27_007"].unique()

In [ ]:
machida_ep.to_csv("data/tmp.csv",columns=['A27_003'])

In [ ]:
machida_ep['A27_003']

machida_ele_countにidを追加する。

In [ ]:
s=pd.Series(list(machida_ep['A27_003']))

In [ ]:
s

In [ ]:
machida_ele_count,
#columns=['id', '計児童・生徒数'
machida_ele_count['計児童・生徒数']